In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_gpd_gdf, to_st_gdf
from electoralyze.region.region_abc import RegionABC

In [7]:
region.SA2_2021.metadata.pipe(print)

shape: (2_472, 2)
┌───────────┬────────────────────────────┐
│ SA2_2021  ┆ SA2_2021_name              │
│ ---       ┆ ---                        │
│ i64       ┆ str                        │
╞═══════════╪════════════════════════════╡
│ 305031128 ┆ Newstead - Bowen Hills     │
│ 511041289 ┆ Irwin                      │
│ 211051285 ┆ Wandin - Seville           │
│ 110011188 ┆ Armidale Surrounds - South │
│ 121011687 ┆ Willoughby                 │
│ …         ┆ …                          │
│ 212031458 ┆ Narre Warren South - West  │
│ 307011178 ┆ Tara                       │
│ 316071546 ┆ Diddillibah - Rosemount    │
│ 202021026 ┆ Bendigo Surrounds - South  │
│ 208021181 ┆ Murrumbeena                │
└───────────┴────────────────────────────┘


In [5]:
region.SA2_2021.geometry.with_columns(st.geom("geometry").st.to_wkt()).pipe(print)

shape: (2_472, 2)
┌───────────┬─────────────────────────────────┐
│ SA2_2021  ┆ geometry                        │
│ ---       ┆ ---                             │
│ i64       ┆ str                             │
╞═══════════╪═════════════════════════════════╡
│ 305031128 ┆ POLYGON ((153.040413 -27.44932… │
│ 511041289 ┆ POLYGON ((114.921946 -29.27032… │
│ 211051285 ┆ POLYGON ((145.411636 -37.79203… │
│ 110011188 ┆ POLYGON ((151.135773 -30.28397… │
│ 121011687 ┆ POLYGON ((151.209539 -33.80446… │
│ …         ┆ …                               │
│ 212031458 ┆ POLYGON ((145.279819 -38.06734… │
│ 307011178 ┆ POLYGON ((150.400255 -26.97881… │
│ 316071546 ┆ POLYGON ((152.976155 -26.61877… │
│ 202021026 ┆ POLYGON ((144.421959 -36.81470… │
│ 208021181 ┆ POLYGON ((145.078188 -37.88713… │
└───────────┴─────────────────────────────────┘


In [3]:
import shutil
import tempfile

from shapely import wkt

region_json = {
    "region_b": ["A", "B", "C"],
    "region_b_name": ["Alpha", "Bravo", "Charlie"],
    "extra": ["1", "2", "3"],
    "geometry": [
        "POLYGON ((-1 -1, 0 1, 1 -1, -1 -1))",
        "POLYGON ((-1 -1, 0 1, -2 1, -1 -1))",
        "POLYGON ((1 -1, 0 1, 2 1, 1 -1))",
    ],
}

# Create the GeoDataFrame
gdf = pl.DataFrame(region_json).with_columns(geometry=st.from_wkt("geometry"))


# Create a temporary directory for the shapefile
temp_dir = tempfile.TemporaryDirectory(delete=False)
shapefile_path = f"{temp_dir.name}/data.shp"

# Write to a shapefile
gdf.pipe(to_gpd_gdf).to_file(shapefile_path)

/tmp/ipykernel_32034/2838255810.py:26: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.pipe(to_gpd_gdf).to_file(shapefile_path)
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'region_b_name' to 'region_b_n'
  ogr_write(


In [4]:
# Read back the shapefile with Pyogrio
read_gdf = pyogrio.read_dataframe(shapefile_path)

# Print the read GeoDataFrame
print(read_gdf)

  region_b region_b_n extra                             geometry
0        A      Alpha     1  POLYGON ((-1 -1, 0 1, 1 -1, -1 -1))
1        B      Bravo     2  POLYGON ((-1 -1, -2 1, 0 1, -1 -1))
2        C    Charlie     3     POLYGON ((1 -1, 0 1, 2 1, 1 -1))


In [5]:
from electoralyze.common.functools import classproperty


class RegionMocked(RegionABC):
    @classproperty
    def raw_geometry_file(self) -> str:
        return shapefile_path

In [6]:
RegionMocked.raw_geometry_file

'/tmp/tmpzv20mo3i/data.shp'

In [7]:
RegionMocked._get_geometry_raw().with_columns(st.geom("geometry").st.area())

proj_create: Cannot find proj.db


Extracting...


proj_create: no database context specified


region_b,region_b_n,extra,geometry
str,str,str,f64
"""A""","""Alpha""","""1""",2.0
"""B""","""Bravo""","""2""",2.0
"""C""","""Charlie""","""3""",2.0


In [8]:
# RegionABC.metadata

In [9]:
shutil.rmtree(temp_dir.name)

# Reading fake regions

In [29]:
from electoralyze.common.testing.region_fixture import REGION_A_JSON, read_true_geometry, read_true_metadata
from electoralyze.region.region_abc import REDISTRIBUTE_FILE

In [30]:
from electoralyze.common.constants import REGION_SIMPLIFY_TOLERANCE

REGION_SIMPLIFY_TOLERANCE

0.0001

In [36]:
read_true_geometry("region_b")
read_true_geometry("region_b")

region_b,geometry
str,binary
"""A""","b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0""…"
"""B""","b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\xc0\x00\x00\x00\x00\x00\x00\xf0""…"
"""C""","b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0""…"


In [37]:
read_true_metadata("region_b")
read_true_metadata("region_a")

region_a,region_a_name,extra
str,str,str
"""M""","""Mew""","""1"""
"""N""","""New""","""2"""
"""O""","""Omega""","""3"""
"""P""","""Phi""","""4"""


# Trying to clear cache

In [50]:
from electoralyze.region.region_abc import RegionABC

RegionABC.geometry

FileNotFoundError: [Errno 2] Failed to open local file '/home/andre/git/private/electoralyze/packages/electoralyze/electoralyze/common/../region/regions/None/geometry/None.parquet'. Detail: [errno 2] No such file or directory

In [48]:
region.SA1_2021.geometry.cache_clear()

AttributeError: 'DataFrame' object has no attribute 'cache_clear'